In [ ]:
import pandas as pd
import numpy as np
import datetime
df = pd.read_csv("excluded_df.csv")
df2 = pd.read_csv("included_df.csv")

df=  df[['BASE_CUSTOMER_ID', 'req_brand_code_ratio',"REQ_TOPMODEL_CODE"]]
df2.drop([ 'req_brand_code_ratio','req_topmodel_code_ratio'], 1, inplace = True)

df_union = pd.merge(df2, df, on = "BASE_CUSTOMER_ID")
df_union.drop(["Age"], 1, inplace = True)


def serial_date_to_string(srl_no):
    new_date = datetime.datetime(2021,5,31,0,0) - datetime.timedelta(srl_no)
    return new_date.strftime("%Y-%m-%d")
test_df = pd.read_csv("included_df.csv")
old_df = pd.read_csv("excluded_df.csv")
old_df=  old_df[['BASE_CUSTOMER_ID', 'req_brand_code_ratio',"REQ_TOPMODEL_CODE"]]
test_df.drop(['req_brand_code_ratio','req_topmodel_code_ratio'], 1, inplace = True)

test_df = pd.merge(test_df, old_df, on = "BASE_CUSTOMER_ID")
test_df.drop(["Age"],1 ,inplace = True)
df_union.drop(["SALESFILE_ID", "SF_CREATE_DATE", "VEHICLE_ID"], 1,inplace = True)
df_union.drop(["FK_RELATION_STATUS_ID"],1 , inplace = True)
test_df.drop(["SALESFILE_ID", "SF_CREATE_DATE", "VEHICLE_ID"], 1,inplace = True)
test_df.drop(["FK_RELATION_STATUS_ID"],1 , inplace = True)

new_idea = pd.read_csv("repair_extr_features_excluded.csv")
new_idea_test = pd.read_csv("repair_extr_features_included.csv")
df_union = df_union.merge(new_idea, on = "BASE_CUSTOMER_ID")


In [ ]:
def create(hyperparams):
    """Create LGBM Classifier for a given set of hyper-parameters."""
    model = LGBMClassifier(**hyperparams)
    return model

def fit(model, X, y):
    """Simple training of a given model."""
    model.fit(X, y)
    return model

def fit_with_stop(model, X, y, X_val, y_val, esr):
    """Advanced training with early stopping."""
    model.fit(X, y,
              eval_set=(X_val, y_val),
              early_stopping_rounds=esr, 
              verbose=200)
    return model

def evaluate(model, X, y):
    """Compute AUC for a given model."""
    yp = model.predict_proba(X)[:, 1]
    auc_score = roc_auc_score(y, yp)
    return auc_score

def kfold_evaluation(X, y, k, hyperparams, esr=100):
    """Run a KFlod evaluation."""
    scores = []
    
    print(f"\n------ {k}-fold evaluation -----")
    print(hyperparams)
    
    kf = KFold(k)
    for i, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"\n----- FOLD {i} -----")
        
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[test_idx]
        y_val = y.iloc[test_idx]
        
        model = create(hyperparams)
        model = fit_with_stop(model, X_train, y_train, X_val, y_val, esr)
        train_score = evaluate(model, X_train, y_train)
        val_score = evaluate(model, X_val, y_val)
        scores.append((train_score, val_score))
        
        print(f"Fold {i} | Eval AUC: {val_score}")
        
        
    scores = pd.DataFrame(scores, columns=['train score', 'validation score'])
    
    return scores

def kfold_prediction(X, y, X_test, k, hyperparams, esr=100):
    """Make predictions with a bagged model based on KFold."""
    yp = np.zeros(len(X_test))
    
    print(f"\n------ {k}-fold evaluation -----")
    print(hyperparams)
    
    kf = KFold(k)
    for i, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"\n----- FOLD {i} -----")
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[test_idx]
        y_val = y.iloc[test_idx]
        
        model = create(hyperparams)
        model = fit_with_stop(model, X_train, y_train, X_val, y_val, esr)
        yp += model.predict_proba(X_test)[:, 1] / k
    
    return yp

In [ ]:
# Constant
K = 10
X_TEST =  test_df.drop(["BASE_CUSTOMER_ID","target"], 1)
BEST_PARAMS = {'n_estimators': 10000, 'learning_rate': 0.05,
 'metric': 'auc'
}

In [ ]:
# Objective function
def objective(trial):
    # Search spaces
    hyperparams = {
        'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 5, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 64),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
        'cat_smooth' : trial.suggest_int('cat_smooth', 10, 100),
        'cat_l2': trial.suggest_int('cat_l2', 1, 20),
        'min_data_per_group': trial.suggest_int('min_data_per_group', 50, 200),
        "is_unbalance" : True
    }
    
    # Add BEST_PARAMS
    hyperparams.update(BEST_PARAMS)
    
    # Evaluation
    scores = kfold_evaluation(X, Y, K, hyperparams, 100)
    
    return scores['validation score'].mean()

In [ ]:
X = df_union.drop(["BASE_CUSTOMER_ID", "target"], 1)
Y = df_union.target
target=  "target"
def create(hyperparams):
    """Create LGBM Classifier for a given set of hyper-parameters."""
    model = LGBMClassifier(**hyperparams)
    return model

def fit(model, X, y):
    """Simple training of a given model."""
    model.fit(X, y)
    return model

def fit_with_stop(model, X, y, X_val, y_val, esr):
    """Advanced training with early stopping."""
    model.fit(X, y,
              eval_set=(X_val, y_val),
              early_stopping_rounds=esr, 
              verbose=200)
    return model

def evaluate(model, X, y):
    """Compute AUC for a given model."""
    yp = model.predict_proba(X)[:, 1]
    auc_score = roc_auc_score(y, yp)
    return auc_score

def kfold_evaluation(X, y, k, hyperparams, esr=100):
    """Run a KFlod evaluation."""
    scores = []
    
    print(f"\n------ {k}-fold evaluation -----")
    print(hyperparams)
    
    kf = KFold(k)
    for i, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"\n----- FOLD {i} -----")
        
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[test_idx]
        y_val = y.iloc[test_idx]
        
        model = create(hyperparams)
        model = fit_with_stop(model, X_train, y_train, X_val, y_val, esr)
        train_score = evaluate(model, X_train, y_train)
        val_score = evaluate(model, X_val, y_val)
        scores.append((train_score, val_score))
        
        print(f"Fold {i} | Eval AUC: {val_score}")
        
        
    scores = pd.DataFrame(scores, columns=['train score', 'validation score'])
    
    return scores

def kfold_prediction(X, y, X_test, k, hyperparams, esr=100):
    """Make predictions with a bagged model based on KFold."""
    yp = np.zeros(len(X_test))
    
    print(f"\n------ {k}-fold evaluation -----")
    print(hyperparams)
    
    kf = KFold(k)
    for i, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"\n----- FOLD {i} -----")
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[test_idx]
        y_val = y.iloc[test_idx]
        
        model = create(hyperparams)
        model = fit_with_stop(model, X_train, y_train, X_val, y_val, esr)
        yp += model.predict_proba(X_test)[:, 1] / k
    
    return yp

In [ ]:
# Optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=3600*1)

In [ ]:
BEST_PARAMS.update(study.BEST_PARAMS)
#BEST_PARAMS = {'reg_alpha': 2.1557466386247137, 'reg_lambda': 3.859970718309284, 'num_leaves': 343, 
 #'min_child_samples': 32, 'max_depth': 7, 'colsample_bytree': 0.2067123441854414, 'cat_smooth': 35, 
  #'cat_l2': 16, 'min_data_per_group': 90,'learning_rate': 0.005, 'metric': 'auc'}

In [ ]:
test = pd.read_csv("sample_submission-Copy1.csv")
test.rename(columns = {"Id": "BASE_CUSTOMER_ID"}, inplace = True)
test = pd.merge(test, test_df, on = "BASE_CUSTOMER_ID", how = "left")
test["target"] = kfold_prediction(X, Y, test.drop(["BASE_CUSTOMER_ID", "target","Expected"] ,1), 10, BEST_PARAMS, 500)
submission = pd.read_csv("sample_submission-Copy1.csv")
submission.Expected = test.target.values

In [ ]:
submission.to_csv("submission1.csv", index = False)